# Verbless clauses and היה-clauses extrabiblical

This notebook is the extrabiblical equivalent of the Biblical notebook for the היה-clauses and verbless clauses. The extrabiblical encoding scheme presently differs slightly from that of the BHSA. For more information, look [here](https://github.com/ETCBC/extrabiblical).

TF is loaded in the old fashioned way, because presently there is no extrabiblical app.

In [1]:
import sys, os, csv, collections

In [2]:
from tf.fabric import Fabric

DATABASE = '~/github'
XBIB = 'extrabiblical'

TF = Fabric(locations='~/github/extrabiblical/tf/0.2') # , modules=[XBIB], silent=False )

This is Text-Fabric 7.4.8
Api reference : https://annotation.github.io/text-fabric/Api/Fabric/

72 features found and 0 ignored


In [3]:
api = TF.load('''
    otype mother lex typ code function rela det txt prs kind vs vt sp book chapter verse label language
''')

api.loadLog()
api.makeAvailableIn(globals())

  0.00s loading features ...
   |     0.02s B otype                from C:/Users/geitb/github/extrabiblical/tf/0.2
   |     0.01s B book                 from C:/Users/geitb/github/extrabiblical/tf/0.2
   |     0.01s B chapter              from C:/Users/geitb/github/extrabiblical/tf/0.2
   |     0.01s B verse                from C:/Users/geitb/github/extrabiblical/tf/0.2
   |     0.03s B mother               from C:/Users/geitb/github/extrabiblical/tf/0.2
   |     0.03s B lex                  from C:/Users/geitb/github/extrabiblical/tf/0.2
   |     0.03s B typ                  from C:/Users/geitb/github/extrabiblical/tf/0.2
   |     0.01s B code                 from C:/Users/geitb/github/extrabiblical/tf/0.2
   |     0.02s B function             from C:/Users/geitb/github/extrabiblical/tf/0.2
   |     0.03s B rela                 from C:/Users/geitb/github/extrabiblical/tf/0.2
   |     0.02s B det                  from C:/Users/geitb/github/extrabiblical/tf/0.2
   |     0.01s B txt     

[('Computed',
  'computed-data',
  ('C Computed', 'Call AllComputeds', 'Cs ComputedString')),
 ('Features', 'edge-features', ('E Edge', 'Eall AllEdges', 'Es EdgeString')),
 ('Fabric', 'loading', ('ensureLoaded', 'TF', 'ignored', 'loadLog')),
 ('Locality', 'locality', ('L Locality',)),
 ('Misc', 'messaging', ('cache', 'error', 'indent', 'info', 'reset')),
 ('Nodes',
  'navigating-nodes',
  ('N Nodes', 'sortKey', 'sortKeyTuple', 'otypeRank', 'sortNodes')),
 ('Features',
  'node-features',
  ('F Feature', 'Fall AllFeatures', 'Fs FeatureString')),
 ('Search', 'search', ('S Search',)),
 ('Text', 'text', ('T Text',))]

The extrabiblical dataset contains the following texts.

In [12]:
for book in F.otype.s('book'):
    chaps = L.d(book, 'chapter')
    for chap in chaps:
        print(T.bookName(book), F.chapter.v(chap))

1QH 3
1QH 4
1QH 5
1QH 6
1QH 7
1QH 8
1QH 9
1QH 10
1QH 11
1QH 12
1QH 13
1QH 14
1QH 15
1QH 16
1QH 17
1QH 18
1QH 19
1QH 20
1QH 21
1QH 22
1QH 23
1QH 24
1QH 25
1QH 26
1QH 27
1QH 28
1QH 29
1QM 1
1QM 2
1QM 3
1QM 4
1QM 5
1QM 6
1QM 7
1QM 8
1QM 9
1QM 10
1QM 11
1QM 12
1QS 1
1QS 2
1QS 3
1QS 4
1QS 5
1QS 6
1QS 7
Kuntillet_Ajrud 18
Kuntillet_Ajrud 19
Kuntillet_Ajrud 20
Arad 1
Arad 2
Arad 40
Balaam 1
Balaam 2
Ketef_Hinnom 1
Ketef_Hinnom 2
Lachish 3
Lachish 4
Lachish 5
Lachish 6
Mesha_Stela 1
Mesad_Hashavyahu 1
Pirqe 1
Pirqe 2
Pirqe 3
Pirqe 4
Pirqe 5
Pirqe 6
Shirata 1
Shirata 2
Shirata 3
Shirata 4
Shirata 5
Shirata 6
Shirata 7
Shirata 8
Shirata 9
Shirata 10
Siloam 1


Count the number of words in this dataset

In [5]:
n = 0

for wo in F.otype.s('word'):
    n += 1
    
print(n)

39862


In a few helper functions it is calculated whether a clause is a main clause, a subordinate clause (3 different varieties) or undecided.

In [6]:
# %load main_sub
def in_dep_calc(cl):  
      
    in_dep = ''        
    if F.rela.v(cl) == 'ReSu': # is the clause resumptive?
        moth_obj = E.mother.f(cl)[0]
        in_dep = rela_calc(moth_obj)
    else:
        in_dep = rela_calc(cl) # does the clause have a dependent CCR?

    if in_dep == '':
    	words = L.d(cl, 'word') # is there a wayyiqtol?
    	for word in words:
            if F.vt.v(word) == 'wayq':
                in_dep += 'Main'
                        
    if in_dep == '':  # if everything else does not give a result, we look at the CARC
        cl_atoms = L.d(cl, 'clause_atom')
        in_dep = carc_calc(cl_atoms)
        
    return(in_dep)


#####################################

# the feature 'code' plays an important role in detrmining whether a clause is a main or dependent clause
def carc_calc(cl_atoms):
    in_dep_c = ''
    carc = F.code.v(cl_atoms[0])
    if 999 > int(carc) > 499:
        in_dep_c += 'SubAdv'
    elif int(carc) in {0, 999}:
        in_dep_c = 'Main'
    elif 17 > int(carc) > 9:
        in_dep_c += 'SubAdv'
    elif 75 > int(carc) > 50:
        in_dep_c += 'SubAdv'
    elif 168 > int(carc) > 99:
        in_dep_c += 'Main'
    elif 500 > int(carc) > 299:
        in_dep_c += 'Main'
    elif int(carc) in {200, 201}:         
        while F.code.v(cl_atoms[0]) in {200, 201}:
            cl_atoms = E.mother.f(cl_atoms[0])
        carc = F.code.v(cl_atoms[0])
        if 999 > int(carc) > 499:
            in_dep_c += 'SubAdv'
        elif int(carc) in {0, 999}:
            in_dep_c = 'Main'
        elif 17 > int(carc) > 9:
            in_dep_c += 'SubAdv'
        elif 75 > int(carc) > 50:
            in_dep_c += 'SubAdv'
        elif 168 > int(carc) > 99:
            in_dep_c += 'Main'
        elif 500 > int(carc) > 299:
            in_dep_c += 'Main'
        elif int(carc) in {220, 221, 222, 223}:
            in_dep_c += 'Undc'
        
    else:
        in_dep_c += 'Undc'
        
    return(in_dep_c)

###################################################################################################

def rela_calc(cl):
    in_dep_r = ''
    ccr = F.rela.v(cl)
    if ccr in {'Subj', 'Objc', 'Cmpl', 'PreC', 'Voct', 'Frnt'}:
        in_dep_r += 'SubArg'
    elif ccr in {'Attr', 'RgRc', 'Spec'}:
        in_dep_r += 'SubMod'
    elif ccr in {'Adju', 'PrAd'}:
        in_dep_r += 'SubAdv'
    elif ccr == 'Coor':
        moth_obj = E.mother.f(cl)[0]
        if F.otype.v(moth_obj) in {'word', 'phrase'}:
            in_dep_r += 'SubMod'
        else:
            while F.rela.v(moth_obj) == 'Coor':
                moth_obj = E.mother.f(moth_obj)[0]
            ccr = F.rela.v(cl)
            if ccr in {'Subj', 'Objc', 'Cmpl', 'PreC', 'Voct', 'Frnt'}:
                in_dep_r += 'SubArg'
            elif ccr in {'Attr', 'RgRc', 'Spec'}:
                in_dep_r += 'SubMod'
            elif ccr in {'Adju', 'PrAd'}:
                in_dep_r += 'SubAdv'
                
        if in_dep_r == '':
            if F.otype.v(moth_obj) != 'clause':
                in_dep_r += 'SubMod'
            else:
                cl_atoms = L.d(moth_obj, 'clause_atom')
                in_dep_r = carc_calc(cl_atoms)
                
    return(in_dep_r)

############################################################################################

The function mother_clause_calc determines the tense of the mother of that clause(atom).

In [7]:
def mother_tense_calc(cl):
    
    mother_tense = ''
    cl_atoms = L.d(cl, 'clause_atom')
    moth = E.mother.f(cl_atoms[0])
    if len(moth) == 0:
        mother_tense += 'no_mother'
    else: 
        if F.otype.v(moth[0]) in {'word', 'phrase'}:
            mother_tense += F.otype.v(moth[0])
        else:
            cl = L.u(moth[0], 'clause')[0]
            if F.kind.v(cl) == 'NC':
                mother_tense += 'nominal'
            elif F.kind.v(cl) == 'WP':
                mother_tense += 'no_pred'
                        
            else:
                phrases = L.d(cl, 'phrase')
                pred = False
                prec = False
                for phr in phrases:
                    if F.function.v(phr) in {'Pred', 'PreS', 'PreO'}:
                        pred = True
                        pred_phr = phr
                    elif F.function.v(phr) in {'PreC', 'PtcO'}:
                        prec = True
                        prec_phr = phr
                if pred == True:
                    words = L.d(pred_phr, 'word')
                    for word in words:
                        if F.sp.v(word) == 'verb':
                            mother_tense += F.vt.v(word)
                elif prec == True:
                    words = L.d(prec_phr, 'word')
                    for word in words:
                        if F.sp.v(word) == 'verb':
                            mother_tense += F.vt.v(word)
                            
    return(mother_tense)

A list with all the different phrase functions is created (in this case copied from the biblical data). This is used in the main program.

In [8]:
extra_cl_funcs = ['Adju', 'Cmpl', 'Conj', 'EPPr', 'ExsS', 'Exst', 'Frnt', 'IntS', 'Intj', 'Loca', 'ModS', 'Modi', 'NCoS', 'NCop', 'Nega', 'Objc', 'PrAd', 'PrcS', 'PreO', 'PreS', 'PtcO', 'Ques', 'Rela', 'Supp', 'Time', 'Voct', 'Unkn']

Of each clause it it determined whether it is a relevant clause and its information is stored in the dict cop_clauses.

In [9]:
cl_list = []
cl_dict = {}

for cl in F.otype.s('clause'):
    feat_list = []
    cl_kind = F.kind.v(cl)
    if cl_kind in {'NC', 'VC'}: 
        phrases = L.d(cl, 'phrase')
        phr_funcs = [F.function.v(phr) for phr in phrases]
        
        if 'Subj' in phr_funcs and 'PreC' in phr_funcs:
            cl_type = ''
            if cl_kind == 'VC' and 'Pred' in phr_funcs:
                cl_type += 'hyh'
            elif cl_kind == 'NC':
                cl_type += 'nom'
                
            if cl_type in {'hyh', 'nom'}:
                
                # collect basic info about clause
                feat_list.append(cl_type) # type of clause: hyh or nom
                feat_list.append(str(cl)) # clause id
                
                bo, ch, ve = T.sectionFromNode(cl)
                feat_list.append(bo) # book
                feat_list.append(str(ch)) # chapter
                feat_list.append(str(ve)) # verse
                
                feat_list.append(F.typ.v(cl)) # clause type of database
                feat_list.append(F.rela.v(cl) + '_') # clause relation
                
                # eg if book is '1_Samuel', add 'Samuel' to feat_list, relevant for biblical data only
                feat_list.append(bo)
                
                # ebh_lbh
                if bo in {'1QM', '1QH', '1QS'}:
                    feat_list.append('qumranic')
                elif bo in {'Shirata', 'Pirqe'}:
                    feat_list.append('rabbinic')
                else:
                    feat_list.append('epigraphic')
                    
                # genre
                if bo in {'1QH', 'Shirata'}:
                    feat_list.append('poetry')
                else:
                    feat_list.append('prose')
                
                #feat_list.append('prose') #genre
                feat_list.append(F.txt.v(cl)[-1]) # Q, D, N
            
                if phr_funcs.index('Subj') < phr_funcs.index('PreC'): #subj and prec order
                    feat_list.append('SP')
                else:
                    feat_list.append('PS')
                    
                # which conjunction
                phr_types = [F.typ.v(phr) for phr in phrases]
                if 'CP' in phr_types: 
                    conj = phrases[phr_types.index('CP')]
                    words = L.d(conj, 'word')
                    words_lex = '_'.join([F.lex.v(wo) for wo in words])
                    feat_list.append(words_lex)
                else:
                    feat_list.append('no_conj')
                
                # clause length, in verbal clauses the Pred is not counted
                if cl_kind == 'VC':
                    feat_list.append(str(len(phrases) - 1)) 
                else:
                    feat_list.append(str(len(phrases)))
                
                subj_pos = phr_funcs.index('Subj')
                feat_list.append(F.typ.v(phrases[subj_pos])) #phrase type of subject
                feat_list.append(F.det.v(phrases[subj_pos])) #determination of subject
                
                # length of subj in words
                words_subj = L.d(phrases[subj_pos], 'word') # length of subject in words
                subj_len = len(words_subj)
                
                # the following is uncommented in the biblical data file, the suffix is not counted as word automatically there
                #for wo in words_subj:                    
                #    if not F.prs.v(wo) in {'absent', 'n/a'}:
                #        subj_len += 1
                feat_list.append(str(subj_len))
                
                prec_pos = phr_funcs.index('PreC')
                feat_list.append(F.typ.v(phrases[prec_pos])) #phrase type of prec
                feat_list.append(F.det.v(phrases[prec_pos])) #determination of prec
                
                # length of prec in words
                words_prec = L.d(phrases[prec_pos], 'word')
                prec_len = len(words_prec)
                
                # just as with the subject, uncomment the following for biblical data
                #for wo in words_prec:                    
                #    if not F.prs.v(wo) in {'absent', 'n/a'}:
                #        prec_len += 1
                feat_list.append(str(prec_len))
            
                # main or subordinate clause
                feat_list.append(in_dep_calc(cl))
                                     
                # negation in clause
                phr_types = [F.typ.v(phr) for phr in phrases]
                if 'NegP' in phr_types:
                    feat_list.append('neg')
                else:
                    feat_list.append('non_neg')
                
                # aramaic or hebrew
                words = L.d(cl, 'word')
                feat_list.append(F.language.v(words[0]))
                
                # collect info about other phrases in clause (present (1) or absent (0))
                for item in extra_cl_funcs:   
                    if item in phr_funcs:
                        feat_list.append('1')
                    else:
                        feat_list.append('0')
                
                # tense ot the mother of the clause_atom
                feat_list.append(mother_tense_calc(cl))
                
                cl_list.append(cl)
                cl_dict[cl] = feat_list                

Information is saved in a csv file.

In [11]:
csvh = open(r"hyh_nom_xbib.csv", "w")

header = ['cl_type', 'cl_id', 'book', 'chapter', 'verse', 'clause_type_etcbc', 'clause_rela', 'book2', 'ebh_lbh', 'genre', 'txt_type', 's_p_order', 'CP', 'cl_len', 'subj_type', 'subj_det', 'subj_len', 'pc_type', 'pc_det', 'pc_len', 'main_sub', 'nega', 'language']

for item in extra_cl_funcs:
    header.append(item)
header.append('mother')

csvh.write('{}\n'.format(','.join(header)))

for key in cl_list:
    features = cl_dict[key]
    csvh.write('{}\n'.format(','.join(features)))
    
csvh.close()